# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,machico,32.7000,-16.7667,27.85,75,75,2.06,PT,1691430413
1,1,forster,-32.1667,152.5167,8.87,92,54,2.21,AU,1691430414
2,2,bredasdorp,-34.5322,20.0403,10.99,85,0,1.71,ZA,1691430414
3,3,alice springs,-23.7000,133.8833,6.75,76,2,0.00,AU,1691430223
4,4,yellowknife,62.4560,-114.3525,18.29,48,75,4.63,CA,1691430170


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
#%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    size = "Humidity",
    frame_width = 800,
    frame_height = 600, 
    scale = 0.5,
    color = "City"
)

# Display the map
map_plot

C:\Users\Stefan Kniejski\anaconda3\envs\dev\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
min_temp_s = city_data_df["Max Temp"] > 22
max_temp_s = city_data_df["Max Temp"] < 26
wind_speed_s = city_data_df["Wind Speed"] > 5

In [8]:
print(sum(min_temp_s))
print(sum(max_temp_s))
print(sum(wind_speed_s))

320
365
142


In [9]:
# A max temperature lower than 26 degrees but higher than 22

# Wind speed more than 5 m/s

# Narrow down cities that fit criteria and drop any results with null values
# Max temp

city_temp_range_df = city_data_df[min_temp_s & max_temp_s & wind_speed_s]
city_temp_range_df

# Drop any rows with null values & Display sample data
clean_city_temp_range_df = city_temp_range_df.dropna()
clean_city_temp_range_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
69,69,port mathurin,-19.6833,63.4167,22.50,65,86,10.88,MU,1691430171
103,103,west island,-12.1568,96.8225,25.99,83,40,6.69,CC,1691430470
125,125,nova sintra,14.8667,-24.7167,22.26,77,89,6.87,CV,1691430482
212,212,murphysboro,37.7645,-89.3351,25.06,69,100,5.14,US,1691430532
228,228,haiku-pauwela,20.9219,-156.3051,25.70,79,20,8.23,US,1691430542
240,240,hobyo,5.3505,48.5268,25.14,81,77,12.71,SO,1691430550
283,283,waingapu,-9.6567,120.2641,22.72,86,1,5.05,ID,1691430575
287,287,luanda,-8.8368,13.2343,23.00,78,0,5.66,AO,1691430457
294,294,bhikangaon,21.8667,75.9500,25.13,79,100,7.55,IN,1691430582
300,300,hadibu,12.6500,54.0333,25.84,79,34,8.62,YE,1691430585


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_city_temp_range_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
69,69,port mathurin,-19.6833,63.4167,22.50,65,86,10.88,MU,1691430171,
103,103,west island,-12.1568,96.8225,25.99,83,40,6.69,CC,1691430470,
125,125,nova sintra,14.8667,-24.7167,22.26,77,89,6.87,CV,1691430482,
212,212,murphysboro,37.7645,-89.3351,25.06,69,100,5.14,US,1691430532,
228,228,haiku-pauwela,20.9219,-156.3051,25.70,79,20,8.23,US,1691430542,
240,240,hobyo,5.3505,48.5268,25.14,81,77,12.71,SO,1691430550,
283,283,waingapu,-9.6567,120.2641,22.72,86,1,5.05,ID,1691430575,
287,287,luanda,-8.8368,13.2343,23.00,78,0,5.66,AO,1691430457,
294,294,bhikangaon,21.8667,75.9500,25.13,79,100,7.55,IN,1691430582,
300,300,hadibu,12.6500,54.0333,25.84,79,34,8.62,YE,1691430585,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 10000
limit = 1
categories = "accommodation.hotel"

#create a params dictionary
params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key     
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lng = hotel_df.loc[index, "Lng"] 
    lat = hotel_df.loc[index, "Lat"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    print(name_address.url)
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
https://api.geoapify.com/v2/places?categories=accommodation.hotel&limit=1&apiKey=7f0f65a63a1448d1b1d4238d2c20705d&filter=circle%3A63.4167%2C-19.6833%2C10000&bias=proximity%3A63.4167%2C-19.6833
port mathurin - nearest hotel: Escale Vacances
https://api.geoapify.com/v2/places?categories=accommodation.hotel&limit=1&apiKey=7f0f65a63a1448d1b1d4238d2c20705d&filter=circle%3A96.8225%2C-12.1568%2C10000&bias=proximity%3A96.8225%2C-12.1568
west island - nearest hotel: Cocos Village Bungalows
https://api.geoapify.com/v2/places?categories=accommodation.hotel&limit=1&apiKey=7f0f65a63a1448d1b1d4238d2c20705d&filter=circle%3A-24.7167%2C14.8667%2C10000&bias=proximity%3A-24.7167%2C14.8667
nova sintra - nearest hotel: Residência Ka Dencho
https://api.geoapify.com/v2/places?categories=accommodation.hotel&limit=1&apiKey=7f0f65a63a1448d1b1d4238d2c20705d&filter=circle%3A-89.3351%2C37.7645%2C10000&bias=proximity%3A-89.3351%2C37.7645
murphysboro - nearest hotel: Holiday Inn Express
https:/

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
69,69,port mathurin,-19.6833,63.4167,22.50,65,86,10.88,MU,1691430171,Escale Vacances
103,103,west island,-12.1568,96.8225,25.99,83,40,6.69,CC,1691430470,Cocos Village Bungalows
125,125,nova sintra,14.8667,-24.7167,22.26,77,89,6.87,CV,1691430482,Residência Ka Dencho
212,212,murphysboro,37.7645,-89.3351,25.06,69,100,5.14,US,1691430532,Holiday Inn Express
228,228,haiku-pauwela,20.9219,-156.3051,25.70,79,20,8.23,US,1691430542,Inn At Mama's Fish House
240,240,hobyo,5.3505,48.5268,25.14,81,77,12.71,SO,1691430550,No hotel found
283,283,waingapu,-9.6567,120.2641,22.72,86,1,5.05,ID,1691430575,HOTEL SANDLEWOOD
287,287,luanda,-8.8368,13.2343,23.00,78,0,5.66,AO,1691430457,Hotel Forum
294,294,bhikangaon,21.8667,75.9500,25.13,79,100,7.55,IN,1691430582,No hotel found
300,300,hadibu,12.6500,54.0333,25.84,79,34,8.62,YE,1691430585,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
#%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600, 
    scale = 0.5,
    color = "City"
)

# Display the map
hotel_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)